In [1]:
import numpy as np
import pandas as pd

In [2]:
flow_rates = pd.read_csv("../data/flow_rates.csv")
flow_apc = pd.read_csv("../data/flow_apc.csv")

ratesMale = flow_rates[flow_rates['sex'] == 'male'].reset_index(drop=True)
apcMale = flow_apc[flow_apc['sex'] == 'male'].reset_index(drop=True)

rates = np.array(ratesMale.iloc[:, 2:])
apc = np.array(apcMale.iloc[:, 2:])

In [3]:
from scripts import pmsltsrc as pms

model = pms.pmsltModel()

In [4]:
baseflows = ['n_s', 's_rs', 'n_v', 'v_rv', 's_vrs', 'v_s']
ages = [0, 14, 15, 16, 18, 21, 24, 28, 33, 40, 50, 60, 70, 90, 109]

In [5]:
flows = model.flows

In [6]:
computed = ['n_s', 's_rs', 'n_v', 'v_rv', 's_vrs', 'v_s', 'age', 'v_sv', 's_sv',
       'sv_s', 'sv_vrs', 'vrs_s', 'vrs_sv', 'vrs_rv', 'sv_rs']

noncomputed = [i for i in flows if i not in computed]

In [7]:
interactions = pd.read_csv("../data/flowInteractions.csv")

interflows = interactions.columns[1:]

tempflows = pd.DataFrame(np.ones((98, 7)), columns=interflows)

tempflows["agecategory"] = [i for i in range(110) if i not in interactions["agecategory"].values]

interactions = pd.concat([interactions, tempflows], axis=0).sort_values(by="agecategory").reset_index(drop=True)

interactions.loc[interactions["agecategory"] > 25, "v_sv from n_s "] = 2

In [8]:
empty = pd.DataFrame(np.empty((95, 6)), columns=baseflows)
empty["age"] = [i for i in np.arange(0, 110) if i not in ages]

In [9]:
def process(inpArray: np.ndarray):
    rates = pd.DataFrame(inpArray[:90].reshape(15, 6), columns=baseflows)
    apc = pd.DataFrame(inpArray[90:].reshape(15, 6), columns=baseflows)

    rates["age"] = ages
    apc["age"] = ages

    rates = pd.concat([rates, empty], ignore_index=True).sort_values(by="age").reset_index(drop=True)
    apc = pd.concat([apc, empty], ignore_index=True).sort_values(by="age").reset_index(drop=True)

    rates = rates.interpolate(method='linear', axis=0)
    apc = apc.interpolate(method='linear', axis=0)

    for interflow in interflows:
        flow1 = interflow.split("from")[1].strip()
        flow2 = interflow.split("from")[0].strip()

        rates[flow2] = rates[flow1] * interactions[interflow]
        apc[flow2] = apc[flow1].copy()

    rates['sv_rs'] = rates[['s_rs', 'v_rv']].min(axis=1)
    apc['sv_rs'] = apc[['s_rs', 'v_rv']].min(axis=1)

    rates[noncomputed] = flow_rates[noncomputed]
    apc[noncomputed] = flow_apc[noncomputed]

    rates = np.array(rates.iloc[:, 1:])
    apc = np.array(apc.iloc[:, 1:])

    return (rates, apc)

In [10]:
def costFunc(inpArray):

    n = inpArray.shape[0]

    outs = [process(inpArray[i, :]) for i in range(n)]
    nrates = [outs[i][0] for i in range(n)]
    napc = [outs[i][1] for i in range(n)]
    outs = [model.pmslt(nrates[i], napc[i]) for i in range(n)]

    j = [score for (score, _) in outs]

    return j

In [11]:
def costFunc_single(inpArray):
    (rates, apc) = process(inpArray)
    score, _ = model.pmslt(rates, apc)
    return score

In [12]:
apc_lower = np.array([-0.25, 0, -0.25, -0.25, -0.25, -0.25]*15)
apc_upper = np.array([0, 0.25, 0.25, 0.25, 0.25, 0.25]*15)

In [13]:
rates_upper = pd.read_csv("../data/flow_rates_upper.csv")
rates_upper = rates_upper[rates_upper['sex'] == 'male'].drop(columns=['sex']).reset_index(drop=True)

rates_upper.loc[rates_upper['agecategory'] == 19, 'agecategory'] = 21

emptyUpper = np.empty((10, 6))
emptyUpper[:] = np.nan
emptyUpper = pd.DataFrame(emptyUpper, columns=baseflows)
emptyUpper["agecategory"] = [i for i in ages if i not in rates_upper["agecategory"].values]

rates_upper = pd.concat([rates_upper, emptyUpper], ignore_index=True).sort_values(by="agecategory").reset_index(drop=True)

rates_upper.ffill(inplace=True)

In [14]:
rates_lower = np.array(np.zeros((9, 6))).flatten()
rates_lower = np.concatenate((rates_lower, np.array([0, 0, 0, 0.01, 0, 0]*6)))

rates_upper = np.array(rates_upper.iloc[:, 1:]).flatten()

In [15]:
constraints = (np.concatenate((rates_lower, apc_lower)), np.concatenate((rates_upper, apc_upper)))

### Swarm optimization

In [15]:
import pyswarms as ps

In [16]:
optimizer = ps.global_best.GlobalBestPSO(n_particles=50, 
                                         dimensions=180,    
                                         options={'c1': 0.5, 'c2': 0.3, 'w': 0.9},
                                         bounds=constraints)

In [17]:
#cost, args = optimizer.optimize(costFunc, iters=100)

2024-07-18 00:55:24,020 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/100, best_cost=6.26e+4c:\Users\orect\mambaforge\envs\popint\lib\site-packages\pyswarms\backend\handlers.py:387: RuntimeWarning: invalid value encountered in remainder
  new_pos[greater_than_bound] = lb[greater_than_bound] + np.mod(
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=6.26e+4
2024-07-18 02:38:55,583 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 62586.453805968915, best pos: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.01758242  0.          0.14520186  0.          0.          0.
  0.0494819   0.          0.293785    0.          0.23274952  0.29996152
  0.11306908  0.          0.05703698  0.          0.23780251  0.04586907
  0.1301143   0.14389011  0.3030244   0.0842512   0.2452

In [23]:
#costFunc_single(args)

np.float64(62586.453805968915)

In [25]:
#rates, apc = process(args)

In [33]:
#np.savetxt("../data/outputs/rates_PSO_50p_100i.csv", rates, delimiter=",")
#np.savetxt("../data/outputs/apc_PSO_50p_100i.csv", apc, delimiter=",")

In [31]:
#score = model.pmslt(rates, apc)

In [41]:
argsRates = pd.DataFrame(args[:90].reshape(15, 6), columns=baseflows)
argsRates["age"] = ages
argsApc = pd.DataFrame(args[90:].reshape(15, 6), columns=baseflows)
argsApc["age"] = ages

In [42]:
argsRates

,n_s,s_rs,n_v,v_rv,s_vrs,v_s,age
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14
2,0.017582,0.000000,0.145202,0.000000,0.000000,0.000000,15
3,0.049482,0.000000,0.293785,0.000000,0.232750,0.299962,16
4,0.113069,0.000000,0.057037,0.000000,0.237803,0.045869,18
5,0.130114,0.143890,0.303024,0.084251,0.245261,0.225004,21
6,0.102588,0.123946,0.058691,0.073797,0.271309,0.189596,24
7,0.000000,0.012449,0.245933,0.096442,0.058568,0.169163,28
8,0.000000,0.143430,0.049390,0.125830,0.290066,0.334415,33
9,0.000000,0.143396,0.175750,0.130697,0.186558,0.204687,40
